In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Load the dataset using Pandas
data = pd.read_csv('diabetes.csv')

x = data.iloc[:,0:-1].values
y_string = list(data.iloc[:,-1])

In [3]:
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [4]:
y = np.array(y_int, dtype = 'float64')

In [5]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [6]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [7]:
class Dataset(Dataset):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(x)

In [8]:
dataset = Dataset(x,y)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                   batch_size=32,
                                   shuffle=True)


In [10]:
# Let's have a look at the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


In [11]:
# Now let's build the above network
class Model(nn.Module):
    def __init__(self, input_features,output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [12]:
net = Model(7,1)

criterion = torch.nn.BCELoss(size_average=True)   

optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

C:\Users\Nouman Rasheed\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [13]:
epochs = 200
for epoch in range(epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        
        # Feed Forward
        outputs = net(inputs)
        
        #output = net.forward(inputs)
        #loss calculation
        
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()
        
    output = (outputs>0.5).float()
    accuracy = (output == labels).float().mean()
    
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1, epochs, loss, accuracy))  

Epoch 1/200, Loss: 0.865, Accuracy: 0.406
Epoch 2/200, Loss: 0.848, Accuracy: 0.406
Epoch 3/200, Loss: 0.818, Accuracy: 0.406
Epoch 4/200, Loss: 0.783, Accuracy: 0.406
Epoch 5/200, Loss: 0.747, Accuracy: 0.406
Epoch 6/200, Loss: 0.717, Accuracy: 0.406
Epoch 7/200, Loss: 0.694, Accuracy: 0.406
Epoch 8/200, Loss: 0.680, Accuracy: 0.594
Epoch 9/200, Loss: 0.673, Accuracy: 0.594
Epoch 10/200, Loss: 0.671, Accuracy: 0.594
Epoch 11/200, Loss: 0.673, Accuracy: 0.594
Epoch 12/200, Loss: 0.678, Accuracy: 0.594
Epoch 13/200, Loss: 0.682, Accuracy: 0.594
Epoch 14/200, Loss: 0.687, Accuracy: 0.594
Epoch 15/200, Loss: 0.690, Accuracy: 0.594
Epoch 16/200, Loss: 0.691, Accuracy: 0.594
Epoch 17/200, Loss: 0.691, Accuracy: 0.594
Epoch 18/200, Loss: 0.690, Accuracy: 0.594
Epoch 19/200, Loss: 0.687, Accuracy: 0.594
Epoch 20/200, Loss: 0.684, Accuracy: 0.594
Epoch 21/200, Loss: 0.679, Accuracy: 0.594
Epoch 22/200, Loss: 0.675, Accuracy: 0.594
Epoch 23/200, Loss: 0.671, Accuracy: 0.594
Epoch 24/200, Loss: 